# Aula 7 - Web Scraping 

## Nossa agenda

1.  Definição
1.  HTML e Scraping com o pacote BeautifulSoup
1.  HTTP, APIs e o pacote Requests
1.  Escrita de arquivos e CSV

## Me manda uma mensagem

#wilsontayar no [slack](https://terahq.slack.com)

/wilsontayar no [github](https://github.com/wilsontayar)

@wilsontayar no [twitter](https://twitter.com/wilsontayar)

----

## O que é Web Scraping

Web Scraping, ou **Web Data Extraction**, é uma técnica onde dados são **extraídos de fontes da web e salvas em seu computador** ou em um banco de dados.

Nesta aula vamos ver como podemos extrair dados de páginas da web e de APIs.

Também veremos como podemos armazená-las e consumi-las em uma planilha.

---

## O que é HTML

**Hypertext Markup Language**, ou HTML, é uma **forma padrão** para se escrever páginas da web.

Ao contrário do que muitas pessoas falam, **HTML não é uma linguagem de programação**, e sim **uma linguagem de marcação**.

Exemplo de HTML:

```html
<body>
    <div id="first-person" class="person">
        <span class="name">Fulano</span>
        <p class="bio">É um exemplo de pessoa</p>
    </div>
</body>
```

Sempre que você abre uma página na internet, **seu navegador irá fazer o download do HTML da página e interpretá-lo**.

É a partir dessa interpretação que cores, espaçamentos, fontes e bordas irão aparecer.


O **HTML é composto por tags**. Cada **tag tem um propósito** diferente.

Por exemplo, o propósito da tag `body` é iniciar o corpo do HTML, onde todas as outras tags irão existir.

---

As tags podem conter **atributos**. **Atributos ajudam as tags a representar o sentido delas** e o que o navegador deverá fazer ao interpretar aquela tag.

Para o web scraping, **o mais importante é perceber o padrão das tags**.

Por exemplo, imagine que nós queremos extrair a lista de produtos de um site de vendas.

![página de smartphones de uma loja virtual - kabum](images/loja_virtual.PNG)

Agora vamos ver o HTML dessa página

![html de uma loja virtual - kabum](images/loja_virtual_html_1.PNG)

Perceba que cada produto está dentro da `<div class="listagem-box">`

Ao abrirmos esta `div`, percebemos outros padrões no HTML, como grifados na foto abaixo:

![html de uma loja virtual - kabum](images/loja_virtual_html_2.PNG)

*Fonte das fotos: https://www.kabum.com.br/celular-telefone/smartphones*

## Como Funciona o BeautifulSoup

O pacote BeautifulSoup (bs4) foi construido para **realizar web scraping de forma fácil**. Ele transforma o HTML de uma página em um objeto python.

Existem 4 tipos de informação no bs4:
- Tag
- NavigableString
- BeautifulSoup
- Comment

A mais importante para o scraping é a Tag. É nela que estão os dados da página.
O objeto da Tag pode ser acessado da seguinte forma: 


In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup('<div class="produto">Meu produto</div>', 'html.parser')
tag = soup.div

# Nossa div
print(tag.name)

# Trocando div por um span
tag.name = "span"
print(tag)

# Acessando dados de atributos
print(tag['class'])

# Adicionando atributos
tag['id'] = "meu-unico-produto"

print(tag)

# Todos os atributos
print(tag.attrs)


div
<span class="produto">Meu produto</span>
['produto']
<span class="produto" id="meu-unico-produto">Meu produto</span>
{'class': ['produto'], 'id': 'meu-unico-produto'}


---

Para baixarmos um HTML da web, vamos precisar da ajuda de outro pacote.

Vamos estudar o **pacote Requests** um pouco mais a frente, mas por enquanto, só precisamos saber que ele consegue fazer o download de uma página HTML para depois passarmos para o bs4.

No exemplo abaixo estamos indo até a página de uma loja virtual, baixando seu HTML, passando pelo bs4 e realizando o scraping da descriçao de um produto

In [6]:
import requests
from bs4 import BeautifulSoup

page = requests.get('https://www.kabum.com.br/celular-telefone/smartphones').text
soup = BeautifulSoup(page, 'html.parser')

print(soup.find('span', {'class': 'H-subtitulo'}).text)

Smartphone Samsung Galaxy J5 Prime SM-G570M, Quad Core 1.4Ghz, Android 6.0.1,Tela 5, 32GB, 13MP, Leitor Digital, Dual Chip, Desbl - Dourado..


**Podemos fazer isso para todos os produtos, como no exemplo abaixo**

In [8]:
import requests
from bs4 import BeautifulSoup

page = requests.get('https://www.kabum.com.br/celular-telefone/smartphones').text
soup = BeautifulSoup(page, 'html.parser')

produtos = soup.find_all('div', {'class': 'listagem-box'})

for produto in produtos[0:5]:
    print(produto.find('span', {'class': 'H-titulo'}).a.text)
    print(produto.find('div', {'class': 'listagem-preco'}).text)
    print('-------------')

Smartphone Samsung Galaxy J5 Prime SM-G570M, Quad Core 1.4Ghz, Android 6.0.1,Tela 5, 32GB, 13MP, Leitor Digital, Dual Chip, Desbl - Dourado
R$ 646,90
-------------
Smartphone Asus Zenfone Go ZB500KG-1A002BR, Quad Core, Android 5.1, Tela 5´, 8GB, 8MP, 3G, Dual Chip Desbloqueado - Preto
R$ 339,15
-------------
Smartphone Samsung Galaxy J5 Prime SM-G570M, Quad Core 1.4Ghz, Android 6.0.1,Tela 5, 32GB, 13MP, Leitor Digital, Dual Chip, Desbl - Preto
R$ 649,89
-------------
Smartphone Asus Zenfone Go ZB500KG-3G027BR, Quad Core, Android 5.1, Tela 5´, 8GB, 8MP, 3G, Dual Chip Desbloqueado - Dourado
R$ 339,15
-------------
Smartphone Asus Zenfone Go ZB500KG-3H028BR, Quad Core, Android 5.1, Tela 5´, 8GB, 8MP, 3G, Dual Chip Desbloqueado - Prata
R$ 339,15
-------------


## Exercício 1 - Extraindo Dados do CoinMarketCap

Vamos usar requests e BeautifulSoup para o scraping do site https://coinmarketcap.com.

Deveremos extrair os seguintes dados:
- Nome da moeda
- Preço atual
- O valor de mercado total
- O valor atual em circulação
- Percentual de mudança nas últimas 24 horas

Tempo: 15 minutos

Dica: acesse a página pelo seu navegador e veja o código fonte dela

---

In [16]:
import requests
from bs4 import BeautifulSoup

page = requests.get('https://coinmarketcap.com').text
soup = BeautifulSoup(page, 'html.parser')

moedas = [1]

for moeda in moedas[0:5]:
    nome = 'moed'
    preco = '$222'
    valor_mercado = '$555' 
    em_circulacao = '46465'
    percentual_dia = '1.56%'
    print('{0}: {1} ({2} / 24h)'.format(nome, preco, percentual_dia))
    print('Em circulação: {0}'.format(em_circulacao))
    print('Valor de mercado: {0}'.format(valor_mercado))
    print('-------------')

moed: $222 (1.56% / 24h)
Em circulação: 46465
Valor de mercado: $555
-------------


### Algumas Considerações

A prática de web scraping as vezes é bloqueada por alguns sites (amazon, por exemplo). Nesses casos, opte por APIs.

Lembre-se de realizar scraping para tarefas lícitas. Muitos conteúdos na internet possuem copyright e não permitem a publicação do material em outro lugar.

Alguns casos na mídia:

[The Verge - Microsoft ordered to let third parties scrape LinkedIn data](https://www.theverge.com/2017/8/15/16148250/microsoft-linkedin-third-party-data-access-judge-ruling)

[The Atlantic - Aaron's Law: Violating a Site's Terms of Service Should Not Land You in Jail](https://www.theatlantic.com/technology/archive/2013/01/aarons-law-violating-a-sites-terms-of-service-should-not-land-you-in-jail/267247/)

## Como funciona o protocolo HTTP

A internet trafega através do protocolo HTTP, ou **Hypertext Transfer Protocol**. O protocolo HTTP foi criado por Tim Berners-Lee, o criador da web, enquanto ele trabalhava na CERN em 1989 (sua equipe e ele também foram responsáveis pelo HTML).

O HTTP possui **duas entidades básicas**: a **Request** (pedido) e a **Response** (resposta)

Sempre que entramos em uma página, **enviamos uma request para o servidor** do site e **esperamos uma response** com o HTML da página.

Sempre que enviamos uma request devemos especificar um **método**.
A tabela abaixo cita os métodos mais comuns e explica brevemente o objetivo de cada um:

Método     | Objetivo
-----------|-----------
**GET**    | Obter informações
**POST**   | Salvar informações
**PUT**    | Atualizar informações
**DELETE** | Deletar informações

Todos eles são comumente utilizados por APIs. 

O método **GET** é o que seu navegador usa para baixar o HTML da página.

O metódo **POST** também é muito utilizado sempre que você enviar algum formulário em algum site.

---

Todos as respostas possuem, além do conteúdo da resposta, um código de status.

Os códigos de status (status code) seguem um padrão.

Os mais comuns são:

Código  |  Significado
--------|---------------
200     | Ok
3XX     | O servidor vai te redirecionar para outra URL
4XX     | O recurso não foi encontrado, não está mais disponível, etc
5XX     | Algum problema aconteceu no servidor e ele não conseguiu processar a sua request



## Como são formadas as APIs REST

Podemos dizer que uma API é como se fosse uma página da web só que para apenas sistemas acessarem, lerem e interagirem.

Por exemplo, ao acessarmos a API da Jsonplaceholder (https://jsonplaceholder.typicode.com/posts/1), vemos o retorno abaixo, ao invés de uma página comum:

```json
{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}
```

Esse padrão de resposta é o que chamamos de JSON, e este é o tipo de resposta mais comum entre as APIs.

Ele foi feito para que máquinas entendam seus dados e os utilizem como quiser.

---

Outra característica das APIs REST é a divisão dos recursos. Por exemplo:

https://jsonplaceholder.typicode.com/users/1

Ao abrirmos esta URL estamos fazendo um GET para obtermos informações do usuário (/users) com código 1 (/1).
Podemos trocar o código do usuário para buscar outros usuários.

Também podemos pedir todos os usuários digitando:

https://jsonplaceholder.typicode.com/users

## Como funciona o pacote Requests

Como vimos anteriormente, o pacote requests do Python é utilizado para nos comunicarmos com URLs.

Podemos realizar GET, POST, PUT, DELETE e quaisquer outros métodos HTTP com ele.

Veja no exemplo abaixo como podemos fazer isso

In [24]:
import requests

api = requests.get('https://jsonplaceholder.typicode.com/users')

json = api.json()

for user in json:
    print('{0} ({1}) - {2}'.format(user['name'], user['email'], user['website']))

Leanne Graham (Sincere@april.biz) - hildegard.org
Ervin Howell (Shanna@melissa.tv) - anastasia.net
Clementine Bauch (Nathan@yesenia.net) - ramiro.info
Patricia Lebsack (Julianne.OConner@kory.org) - kale.biz
Chelsey Dietrich (Lucio_Hettinger@annie.ca) - demarco.info
Mrs. Dennis Schulist (Karley_Dach@jasper.info) - ola.org
Kurtis Weissnat (Telly.Hoeger@billy.biz) - elvis.io
Nicholas Runolfsdottir V (Sherwood@rosamond.me) - jacynthe.com
Glenna Reichert (Chaim_McDermott@dana.io) - conrad.com
Clementina DuBuque (Rey.Padberg@karina.biz) - ambrose.net


## Exercício 2 - Extraindo Dados do CoinMarketCap via API

Vamos usar requests para o acessar a API do CoinMarketCap.

As instruções da API estão disponíveis em: https://coinmarketcap.com/api/

Deveremos extrair os seguintes dados:
- Nome da moeda
- Preço atual **CONVERTIDO EM REAIS**
- O valor de mercado total
- O valor atual em circulação
- Percentual de mudança nas últimas 24 horas

Também deveremos limitar para que a API traga apenas 5 moedas para trabalharmos

Tempo: 15 minutos

---

In [ ]:
import requests

api = requests.get('API')

json = api.json()

for moeda in json:
    nome = 'moed'
    preco = '$222'
    valor_mercado = '$555' 
    em_circulacao = '46465'
    percentual_dia = '1.56%'
    print('{0}: {1} ({2} / 24h)'.format(nome, preco, percentual_dia))
    print('Em circulação: {0}'.format(em_circulacao))
    print('Valor de mercado: {0}'.format(valor_mercado))
    print('-------------')

## Escrita e leitura de arquivos em Python


Em Python, assim como em outras linguagens de programação, podemos manipular arquivos que estão em nosso disco.

Para isso, utilizamos o `open` para nos trazer um objeto de arquivo.

O `open` precisa de 1 parâmetro, e suporta mais 2 parametros opcionais.

Todo o processo de escrita e leitura de arquivos deve ser feita com muito cuidado. Como são operações mais lentas, você pode acabar travando seu computador se fizer um loop errado enquanto escreve um arquivo.

Por isso, é muito importante se lembrar de SEMPRE fechar o arquivo após o uso dele no Python.

Vamos ver um exemplo:

In [35]:
f = open('arquivo_teste.txt', 'r+')

f.write('Alô? Alguém me escuta?')

print(f.readline())

f.close()

Alô? Alguém me escuta?
